In [6]:
import pandas as pd

from sklearn.metrics import accuracy_score, make_scorer, precision_recall_fscore_support, f1_score
from sklearn.model_selection import StratifiedKFold

import sys
import time

sys.path.insert(1, "../src")
from knn import *
from pca import *

N_SPLITS = 10

train = pd.read_csv("../data/train.csv")

X = train.drop(columns='label').to_numpy()
y = train.label.to_numpy()

## Todo junto (Podemos ir cambiando las cositas para cada exp)

In [ ]:
grid = {
    "k": [2],
    # "k": [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
    "alpha": [10],
    # "alpha": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 153],
    "sizes": [2],
    "folds": [2],
}

skf = StratifiedKFold(n_splits=N_SPLITS)
splits = list(skf.split(X, y))

metrics = dict()

for fold in grid["folds"]:
    skf = StratifiedKFold(n_splits=folds)
    
    for size in grid["sizes"]:
        splits = list(skf.split(X[:size], y[:size]))

        for a in grid["alpha"]:
            _pca = PCA(alpha=a)
            
            for k in grid["k"]:
                    score_values = []
                    time_values = []
                    print(f"Running gridsearch for: k={k} - alpha={a}")

                    print("\tSplits:")
                    for idx, (train_index, test_index) in enumerate(splits):
                        print(f"\t\t{idx} time:", end="\t\t")
                        _x_train, _y_train = X[train_index], y[train_index]
                        _x_test, _y_test = X[test_index], y[test_index]

                        _knn = KNNClassifier(k=k)

                        _st = time.time()

                        _pca.fit(_x_train)
                        _x_train_transformed = _pca.transform(_x_train)
                        _knn.fit(_x_train_transformed, _y_train)
                        _x_test_transformed = _pca.transform(_x_test)
                        _pred = _knn.predict(_x_test_transformed)

                        _et = time.time()

                        print(f"{(_et - _st):.4f}")

                        _metric = accuracy_score(_y_test, _pred)
                        score_values.append(_metric)
                        time_values.append(_et - _st)

                    print("")

                    metrics[f"{k} - {a} - {size}"] = dict()
                    metrics[f"{k} - {a} - {size}"]['score'] = np.mean(score_values)
                    metrics[f"{k} - {a} - {size}"]['time'] = np.mean(time_values)
                    print(np.mean(score_values))

## kNN con PCA

In [ ]:
grid = {
    "k": [2],
    # "k": [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
    "alpha": [10],
    # "alpha": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 153],
}

skf = StratifiedKFold(n_splits=N_SPLITS)
splits = list(skf.split(X, y))

metrics = dict()

for k in grid["k"]:
    for a in grid["alpha"]:
        
        score_values = []
        time_values = []
        print(f"Running gridsearch for: k={k} - alpha={a}")
        
        print("\tSplits:")
        for idx, (train_index, test_index) in enumerate(splits):
            print(f"\t\t{idx} time:", end="\t\t")
            _x_train, _y_train = X[train_index], y[train_index]
            _x_test, _y_test = X[test_index], y[test_index]
            
            _pca, _knn = PCA(alpha=a), KNNClassifier(k=k)
            
            _st = time.time()
            
            _pca.fit(_x_train)
            _x_train_transformed = _pca.transform(_x_train)
            _knn.fit(_x_train_transformed, _y_train)
            _x_test_transformed = _pca.transform(_x_test)
            _pred = _knn.predict(_x_test_transformed)
            
            _et = time.time()
            
            print(f"{(_et - _st):.4f}")
            
            _metric = accuracy_score(_y_test, _pred)
            score_values.append(_metric)
            time_values.append(_et - _st)
        
        print("")
        
        metrics[f"{k} - {a}"] = dict()
        metrics[f"{k} - {a}"]['score'] = np.mean(score_values)
        metrics[f"{k} - {a}"]['time'] = np.mean(time_values)
        print(np.mean(score_values))

## kNN sin PCA

In [ ]:
grid = {
    "k": [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
}

skf = StratifiedKFold(n_splits=N_SPLITS)
splits = list(skf.split(X, y))

metrics = dict()

for k in grid["k"]:
    score_values = []
    f1_score_values = []
    cohen_cappa_values = []
    time_values = []
    print(f"Running gridsearch for: k={k}")

    print("\tSplits:")
    for idx, (train_index, test_index) in enumerate(splits):
        print(f"\t\t{idx} time:", end="\t\t")
        _x_train, _y_train = X[train_index], y[train_index]
        _x_test, _y_test = X[test_index], y[test_index]

        _knn = KNNClassifier(k=k)

        _st = time.time()

        _knn.fit(_x_train, _y_train)
        _pred = _knn.predict(_x_test)

        _et = time.time()

        print(f"{(_et - _st):.4f}")

        _metric = accuracy_score(_y_test, _pred)
        _f1_score = f1_score(_y_test, _pred, average=None)
        score_values.append(_metric)
        f1_score_values.append(_f1_score)
        time_values.append(_et - _st)

    print("")

    metrics[f"{k}"] = dict()
    metrics[f"{k}"]['score'] = np.mean(score_values)
    metrics[f"{k}"]['f1_score'] = np.mean(time_values)
    metrics[f"{k}"]['time'] = np.mean(time_values)
    print(np.mean(score_values))

Running gridsearch for: k=2
	Splits:
		0 time:		739.1977
		1 time:		

In [ ]:
metrics

## Variando el tamaño de los datos de entrada

In [ ]:
grid = {
    "sizes": [2],
}

CHOSEN_K = 0
CHOSEN_ALPHA = 0

skf = StratifiedKFold(n_splits=N_SPLITS)

metrics = dict()

_pca, _knn = PCA(alpha=CHOSEN_ALPHA), KNNClassifier(k=CHOSEN_K)

for size in grid["sizes"]:
    score_values = []
    time_values = []
    splits = list(skf.split(X[:size], y[:size]))
    print(f"Running gridsearch for: k={k} - alpha={a}")
    
    print("\tSplits:")
    for idx, (train_index, test_index) in enumerate(splits):
        print(f"\t\t{idx} time:", end="\t\t")
        _x_train, _y_train = X[train_index], y[train_index]
        _x_test, _y_test = X[test_index], y[test_index]

        _st = time.time()

        _pca.fit(_x_train)
        _x_train_transformed = _pca.transform(_x_train)
        _knn.fit(_x_train_transformed, _y_train)
        _x_test_transformed = _pca.transform(_x_test)
        _pred = _knn.predict(_x_test_transformed)

        _et = time.time()

        print(f"{(_et - _st):.4f}")

        _metric = accuracy_score(_y_test, _pred)
        score_values.append(_metric)
        time_values.append(_et - _st)

    print("")

    metrics[f"{k} - {a} - {size}"] = dict()
    metrics[f"{k} - {a} - {size}"]['score'] = np.mean(score_values)
    metrics[f"{k} - {a} - {size}"]['time'] = np.mean(time_values)
    print(np.mean(score_values))

## Variando K de K-fold

In [ ]:
grid = {
    "folds": [2],
    # "k": [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
}

CHOSEN_K = 0
CHOSEN_ALPHA = 0

metrics = dict()

_pca, _knn = PCA(alpha=CHOSEN_ALPHA), KNNClassifier(k=CHOSEN_K)

for folds in grid["folds"]:
    score_values = []
    time_values = []
    skf = StratifiedKFold(n_splits=folds)
    splits = list(skf.split(X, y))
    print(f"Running gridsearch for: k={k} - alpha={a}")
    
    print("\tSplits:")
    for idx, (train_index, test_index) in enumerate(splits):
        print(f"\t\t{idx} time:", end="\t\t")
        _x_train, _y_train = X[train_index], y[train_index]
        _x_test, _y_test = X[test_index], y[test_index]

        _st = time.time()

        _pca.fit(_x_train)
        _x_train_transformed = _pca.transform(_x_train)
        _knn.fit(_x_train_transformed, _y_train)
        _x_test_transformed = _pca.transform(_x_test)
        _pred = _knn.predict(_x_test_transformed)

        _et = time.time()

        print(f"{(_et - _st):.4f}")

        _metric = accuracy_score(_y_test, _pred)
        score_values.append(_metric)
        time_values.append(_et - _st)

    print("")

    metrics[f"{k} - {a} - {folds}"] = dict()
    metrics[f"{k} - {a} - {folds}"]['score'] = np.mean(score_values)
    metrics[f"{k} - {a} - {folds}"]['time'] = np.mean(time_values)
    print(np.mean(score_values))

In [3]:
metrics["2 - 10"]

{'score': 0.9093809523809524,
 'cohen-kappa': 0.8992820339190011,
 'recall': 210.68125829535683,
 'f1': 0.9093380737769404,
 'confusion-matrix': [[[785, 0, 3, 5, 2, 15, 9, 1, 5, 2],
   [0, 916, 5, 4, 2, 0, 3, 2, 2, 3],
   [3, 2, 790, 9, 4, 3, 3, 10, 11, 0],
   [2, 4, 11, 733, 4, 31, 2, 2, 70, 11],
   [1, 1, 5, 0, 701, 5, 5, 6, 2, 88],
   [12, 3, 0, 21, 4, 669, 11, 2, 27, 10],
   [7, 0, 4, 0, 6, 8, 801, 1, 0, 0],
   [0, 4, 11, 3, 9, 2, 1, 812, 4, 34],
   [3, 2, 8, 39, 7, 15, 8, 1, 717, 13],
   [3, 4, 1, 13, 80, 6, 6, 30, 10, 685]],
  [[801, 0, 2, 1, 0, 13, 3, 1, 2, 3],
   [0, 928, 1, 3, 0, 0, 1, 1, 0, 3],
   [6, 1, 785, 5, 4, 1, 7, 12, 12, 3],
   [7, 3, 17, 739, 0, 20, 2, 3, 68, 11],
   [1, 3, 5, 2, 714, 2, 6, 8, 1, 73],
   [15, 0, 2, 28, 3, 666, 13, 0, 23, 9],
   [3, 0, 3, 0, 4, 15, 798, 0, 2, 2],
   [0, 10, 7, 5, 7, 1, 0, 814, 2, 34],
   [9, 4, 9, 31, 8, 17, 4, 2, 710, 19],
   [0, 3, 2, 7, 66, 10, 1, 22, 13, 713]],
  [[793, 0, 5, 4, 1, 12, 5, 1, 3, 2],
   [0, 920, 4, 1, 1, 3, 1, 4, 2,